In [1]:
# power = 260mW 
# T = 64 C 
# %matplotlib ipympl 
%matplotlib qt 
%load_ext autoreload 
%autoreload 2
from pianoq.lab.asi_cam import ASICam
from pianoq.lab.dmd import DMD
import numpy as np 
from pianoq.misc.mplt import mimshow, mplot
import datetime 
import time 
from pianoq.lab.photon_scan import PhotonScanner
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.photon_counter import PhotonCounter
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.lee_holo.generate_lee import main, make_oam_donut, orthogonal_lee

In [2]:
dmd = DMD()
dmd.set_white()

Connected!
Built project!


In [3]:
# grtn = 50
# dmd.set_grating(grtn)

# qq = 90
# dmd.set_checkerboard(qq)

# target = make_oam_donut(Nx=dmd.Nx, Ny=dmd.Ny, ell=2, r0=0.35, sigma=0.2)
# hologram, _ = orthogonal_lee(target, carrier_f=(1/50, 0), duty_cycle=0.5)
# dmd.set_image(hologram.astype(bool))

amps = np.array([0, 0.5, 1])
dmd.set_1d_macrosteps(amps, step_w=150, tile=8)

mimshow(dmd.cur_image)

(<Figure size 640x480 with 2 Axes>, <Axes: >)

In [15]:
# mimshow(dmd.cur_image)
print(dmd.last_called_setter)

DMD.set_1d_macrosteps(amplitudes=array([0. , 0.5, 1. ]), step_w=150, tile=8)


In [5]:
timeout = 100000
x_motor = ThorlabsKcubeDC(serial_number=27253522, timeout=timeout)
y_motor = ThorlabsKcubeStepper(serial_number=26001271, timeout=timeout)

In [2]:
integration_time = 15
# coin_window = 4e-9 
# ph = PhotonCounter(serial_port='COM8', integration_time=integration_time)
coin_window = 1.2e-9 
# When DMD-plane fiber is 100um 1m. Delays=(0, 500) when both are 50um 2m fibers
tt = ph = QPTimeTagger(integration_time=integration_time, coin_window=coin_window, single_channel_delays=(0, 5000), remote=True, address='132.64.81.123:41101')  

In [13]:
tt.set_integration_time(1)
tt.show_gui()

In [9]:
s1, s2, c = ph.read_interesting()
s1, s2, c = s1.item(), s2.item(), c.item()
acc = 2*s1*s2*coin_window
real_c = c - acc 
std = np.sqrt(c) / np.sqrt(ph.integration_time)
print(f'{c=:.1f}; {real_c=:.1f}+-{std:.1f}')
print(x_motor.get_position())
print(y_motor.get_position())

c=200.7; real_c=18.5+-4.5
10.49948
12.4999992


In [13]:
start_x = 9.5 # 16.5
start_y = 12.0 # 14.75  
x_pixels = 100
y_pixels = 4
pixel_size_x = 0.1
pixel_size_y = 0.1
name = 'steps_3_150_timetagger_z_12_fiber_100um_fiber_50um_260mW_f_none_before_crystal_filter_20nm'

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
dir_path = r'G:\My Drive\Projects\DMD\quantum\Results\Scans'

path = f'{dir_path}\\{timestamp}_{name}.scan'
scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y,
                        run_name=name, is_timetagger=True, coin_window=coin_window, saveto_path=path)

In [14]:
# time.sleep(10)
single1s, single2s, coincidences = scanner.scan(x_motor=x_motor, y_motor=y_motor, ph=tt)

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 405526. Singles2: 149877. Coincidence: 144.40. real_coin: -1.5.
nan issue in timetagger
dur: 16. pix: 0, 1. Singles1: 405129. Singles2: 154979. Coincidence: 149.87. real_coin: -0.8.
dur: 47. pix: 0, 2. Singles1: 405361. Singles2: 159352. Coincidence: 152.40. real_coin: -2.6.
dur: 63. pix: 0, 3. Singles1: 404976. Singles2: 164413. Coincidence: 165.93. real_coin: 6.1.
dur: 79. pix: 0, 4. Singles1: 405214. Singles2: 167354. Coincidence: 164.13. real_coin: 1.4.
dur: 95. pix: 0, 5. Singles1: 405771. Singles2: 173549. Coincidence: 166.87. real_coin: -2.1.
dur: 111. pix: 0, 6. Singles1: 405382. Singles2: 177387. Coincidence: 168.40. real_coin: -4.2.
nan issue in timetagger
dur: 127. pix: 0, 7. Singles1: 405800. Singles2: 181612. Coincidence: 178.60. real_coin: 1.7.
dur: 158. pix: 0, 8. Singles1: 405198. Singles2: 182790. Coincidence: 184.13. real_coin: 6.4.
dur: 174. pix: 0, 9. Singles1: 405186. Singles2: 184756. Coinci

In [27]:
mplot(scanner.result.real_coins.sum(axis=0))

(<Figure size 640x480 with 1 Axes>, <Axes: >)

In [62]:
x_motor.close()
y_motor.close()
ph.close()
dmd.close()

[autoreload of pianoq.lab.dmd failed: Traceback (most recent call last):
  File "c:\Users\owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "c:\Users\owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\Users\owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\extensions\autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\Users\owner\A

## ASI cam 

In [ ]:
cam = ASICam()

In [ ]:
path = r'G:\My Drive\Projects\DMD\quantum\Results\Callibrations'
# cam.set_roi(4300, 2100, 500, 500)
cam.set_exposure(3)
cam.set_roi(None, None, None, None)
cam.set_binning(4)
# cam.show_image(vmin=None, vmax=None)
cam.save_image(rf'{path}\farfield_plane_yes_lens_75mm_f1_200mm.fits')

In [ ]:
cam.close()

In [ ]:
# pump with f=75mm 
mask = np.ones(dmd.shape, dtype=np.uint8)
mask[:200, :] = 0 
mask[700:, :] = 0 
mask[:, :350] = 0 
mask[:, 600:] = 0 
dmd.set_image(mask)

In [ ]:
mask = np.ones(dmd.shape, dtype=np.uint8)
# mask[:200, :] = 0 
# mask[700:, :] = 0 
# mask[:, :350] = 0 
# mask[:, 600:] = 0 
dmd.set_image(mask)

In [ ]:
r = 450
X0 = 0
Y0 = 0
mask = np.zeros(dmd.shape, dtype=np.uint8)
mask[np.sqrt((dmd.XX-X0)**2+(dmd.YY-Y0)**2) < r] = 1 
dmd.set_image(mask)
# mimshow(mask)